In [2]:
# This pytorch notebook has been written by Adib Bazgir and Rama chandra praneeth Madugula for Stanford Flame AI competition.this model utilises edsr baseline architecture with some modificationa and attaching SE block to the residual block to improve the efficiency
# Other versions of this code within tensorflow framework will be developed very soon. More model architectures based on FNO, GAN, etc will be developed base on the same provided dataset.
# This code will be promptly available on github and kaggle public notebook after the final comptetion outcome becomes released!

# install the accelerate package

!pip install accelerate

In [3]:
# import the required libraries

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR,ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize, Compose, ToTensor
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import datetime
import pandas as pd

In [4]:
# wipe the working directory for any output

!rm -rf /kaggle/working

rm: cannot remove '/kaggle/working': Device or resource busy


In [5]:
# directories to inputs and outputs

base_path = "/kaggle/input/2023-flame-ai-challenge/"
working_dir = "/kaggle/working/"
input_path = base_path + "dataset/"
output_path = working_dir + "outputs/"

# create directories for checkpoints and logs
log_dir = output_path + "logs/"
checkpoint_dir = output_path + "ckpt/"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

train_df = pd.read_csv(input_path + "train.csv")
val_df = pd.read_csv(input_path + "val.csv")
test_df = pd.read_csv(input_path + "test.csv")

In [6]:
# increase the rate at which output messages can be displayed

from IPython.display import display, Javascript

# Set the iopub_msg_rate_limit to a higher value
display(Javascript('''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.kernel.execute("config = get_ipython().config");
    Jupyter.notebook.kernel.execute("config.NotebookApp.iopub_msg_rate_limit = 4000.0");
});
'''))

<IPython.core.display.Javascript object>

In [7]:
# model architecture for training

# Define the Squeeze-and-Excitation block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Modify the ResNetBlock to include the SE block
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, num_filters, kernel_size=3):
        super(ResNetBlock, self).__init__()
        self.resnet_block = torch.nn.Sequential(
            *[
                nn.Conv2d(num_filters, num_filters, kernel_size, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(num_filters, num_filters, kernel_size, padding=1),
            ]
        )
        self.se_block = SEBlock(num_filters)  # Add SE block here
        self.input = nn.Sequential()

    def forward(self, x):
        inp = self.input(x)
        x = self.resnet_block(x)
        x = self.se_block(x)  # Apply SE block
        return x + inp


class Model(nn.Module):
    def __init__(
        self, in_channels=4, factor=2, scale=3, num_of_residual_blocks=20, num_filters=64, kernel_size=3, **kwargs
    ):
        super().__init__()
        self.num_of_residual_blocks = num_of_residual_blocks
        self.scale = scale
        self.factor = factor
        self.in_channels = in_channels
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.res_blocks = nn.Sequential(
            *[
                ResNetBlock(
                    in_channels=in_channels,
                    num_filters=num_filters,
                    kernel_size=kernel_size,
                )
            ]
            * num_of_residual_blocks
        )

        self.upsample = nn.Sequential(
            *[
                nn.Conv2d(num_filters, num_filters * (factor**2), kernel_size=kernel_size, padding=1, **kwargs),
                nn.PixelShuffle(upscale_factor=factor),
            ]
            * scale
        )
        self.resnet_input = nn.Conv2d(in_channels, num_filters, kernel_size=1)
        self.output_layer = nn.Conv2d(num_filters, in_channels, kernel_size=3, padding=1)
        self.resnet_out = nn.Conv2d(self.num_filters, self.num_filters, kernel_size=kernel_size, padding=1)

    def forward(self, x):
        x = self.resnet_input(x)
        x_res = self.res_blocks(x)
        x_res = self.resnet_out(x_res)
        out = x + x_res
        out = self.upsample(out)
        return self.output_layer(out)

In [8]:
# add lightning module for pytorch

class FlowFieldModel(LightningModule):
    def __init__(self):
        super().__init__()

        self.model = Model()
        # Loss function
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.loss_fn(outputs, targets)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        val_loss = self.loss_fn(outputs, targets)
        self.log("val_loss", val_loss, sync_dist=True)

    def configure_optimizers(self):
        optimizer = Adam(params=self.parameters(), lr=0.001)
        scheduler = ReduceLROnPlateau(optimizer=optimizer)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }

In [9]:
# load and preprocess custom utilized dataset

class FlowFieldDataset(Dataset):
    def __init__(self, input_path, mode):
        assert mode in ["train", "val", "test"]
        self.mode = mode
        self.csv_file = pd.read_csv(input_path + f"{mode}.csv")
        if mode == "test":
            self.csv_file = pd.read_csv(input_path + f"{mode}.csv")
        self.LR_path = input_path + "flowfields/LR/" + mode
        self.HR_path = input_path + "flowfields/HR/" + mode

        self.mean = np.array([0.24, 28.0, 28.0, 28.0])
        self.std = np.array([0.068, 48.0, 48.0, 48.0])

    def transform(self, x):
        return Compose([ToTensor(), Normalize(self.mean, self.std, inplace=True)])(x)

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        # input
        if self.mode == "test":
            id = self.csv_file["id"][idx]
            rho_i = np.fromfile(self.LR_path + "/" + self.csv_file["rho_filename"][idx], dtype="<f4").reshape(16, 16)
            ux_i = np.fromfile(self.LR_path + "/" + self.csv_file["ux_filename"][idx], dtype="<f4").reshape(16, 16)
            uy_i = np.fromfile(self.LR_path + "/" + self.csv_file["uy_filename"][idx], dtype="<f4").reshape(16, 16)
            uz_i = np.fromfile(self.LR_path + "/" + self.csv_file["uz_filename"][idx], dtype="<f4").reshape(16, 16)
            X = np.stack([rho_i, ux_i, uy_i, uz_i], axis=2)
            return id, self.transform(X)

        rho_i = np.fromfile(self.LR_path + "/" + self.csv_file["rho_filename"][idx], dtype="<f4").reshape(16, 16)
        ux_i = np.fromfile(self.LR_path + "/" + self.csv_file["ux_filename"][idx], dtype="<f4").reshape(16, 16)
        uy_i = np.fromfile(self.LR_path + "/" + self.csv_file["uy_filename"][idx], dtype="<f4").reshape(16, 16)
        uz_i = np.fromfile(self.LR_path + "/" + self.csv_file["uz_filename"][idx], dtype="<f4").reshape(16, 16)
        # output
        rho_o = np.fromfile(self.HR_path + "/" + self.csv_file["rho_filename"][idx], dtype="<f4").reshape(128, 128)
        ux_o = np.fromfile(self.HR_path + "/" + self.csv_file["ux_filename"][idx], dtype="<f4").reshape(128, 128)
        uy_o = np.fromfile(self.HR_path + "/" + self.csv_file["uy_filename"][idx], dtype="<f4").reshape(128, 128)
        uz_o = np.fromfile(self.HR_path + "/" + self.csv_file["uz_filename"][idx], dtype="<f4").reshape(128, 128)
        X = np.stack([rho_i, ux_i, uy_i, uz_i], axis=2)
        Y = np.stack([rho_o, ux_o, uy_o, uz_o], axis=2)
        return self.transform(X), self.transform(Y)

In [10]:
if __name__ == '__main__':
    # Set hyperparameters here
    num_epochs = 300
    learning_rate = 1e-3
    batch_size = 32
    train_dataset = FlowFieldDataset(input_path=input_path, mode="train")
    val_dataset = FlowFieldDataset(input_path=input_path, mode="val")
    test_dataset = FlowFieldDataset(input_path=input_path, mode="test")

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False, pin_memory=False)

    model =FlowFieldModel()

    # Define a logger for TensorBoard
    logger = TensorBoardLogger("logs", name="tensorboard")

    # Define a ModelCheckpoint callback to save the best model
    checkpoint_callback = ModelCheckpoint(
        dirpath=checkpoint_dir,
        filename="ckpt",
        save_top_k=1,
        monitor="val_loss",
        mode="min",
    )

    trainer = Trainer(
        deterministic=True,
        max_epochs=num_epochs,
        devices=2,
        accelerator="gpu",
         # Set to 0 for CPU or specify GPU device IDs
        logger=logger,
        log_every_n_steps=5,
        callbacks=[checkpoint_callback],
    )


    trainer.fit(model, train_dataloader, val_dataloader)

In [11]:
# perform the inference for the trained model

progress_bar = tqdm(range(len(test_dataloader)))
predictions = {}
ids = []
for idx, batch in enumerate(test_dataloader):
    id, inputs = batch
    outputs = model(inputs)
    outputs = outputs.permute(0, 2, 3, 1)
    predictions[idx] = outputs.cpu().detach().numpy().flatten(order="C").astype(np.float32)
    ids.append(id.cpu().detach().numpy()[0])
    progress_bar.set_description(f"test prediction: {idx}")
    progress_bar.update(1)
progress_bar.close()

df = pd.DataFrame.from_dict(predictions).T
df["id"] = ids
# move id to first column
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
# reset index
df = df.reset_index(drop=True)

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the date and time as a string
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

# Define the filename with the formatted date and time
filename = f"predictions_{formatted_datetime}.csv"

# Save the DataFrame to the CSV file
df.to_csv(filename, index=False)

test prediction: 172: 100%|██████████| 173/173 [00:13<00:00, 12.58it/s]


In [12]:
# save the model weights

torch.save(model.state_dict(), "model_weights0.0087.pth")

In [13]:
# save the full model including the weights

torch.save(model,"model_full0.0087.pth")

In [14]:
# load the model weights

model.load_state_dict(torch.load('model_weights0.0087.pth'))

<All keys matched successfully>

In [15]:
# load the full model

model = torch.load('model_full0.0087.pth')